In [ ]:
import sklearn as sk
import streamlit as st
import pandas as pd
import numpy as np
import sklearn as sk
import xgboost as xgb
import tensorflow as tf
import keras as ks
import seaborn as sns
import plotly.express as px

filepath = "https://raw.githubusercontent.com/lizh1988/portfolio/main/Conc%20strength%20predictor/concrete_data.csv"

df = pd.read_csv(filepath, sep = ',')
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.rename(columns={'blast_furnace_slag':'slag',
         'fly_ash':'ash', 
         'superplasticizer':'sp',
         'coarse_aggregate':'coarse',
         'concrete_compressive_strength':'strength',
         'fine_aggregate ':'fine'}, inplace=True)
df.head()

In [ ]:
fig = sns.boxplot(data=df)
fig

In [ ]:
scaler=sk.preprocessing.StandardScaler()

x = df.drop(['strength'],axis=1)

x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(x ,df['strength'], test_size=0.2, random_state=88)

x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)

In [ ]:
#le = sk.preprocessing.LabelEncoder()
#y_train=le.fit_transform(y_train)

reg=xgb.XGBRegressor()
reg.fit(x_train,y_train)

In [ ]:
kfold=sk.model_selection.KFold(n_splits=10, shuffle=True)

score=sk.model_selection.cross_val_score(reg, x_train, y_train,cv=kfold)
score.mean()

In [ ]:
ypred=reg.predict(x_test)
mse=sk.metrics.mean_squared_error(y_test,ypred)
r2=sk.metrics.r2_score(y_test,ypred)
r2

In [ ]:
parameters = {'device':['cuda'],
                'nthread':[2], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7,8,9],
              'min_child_weight': [4],
              'verbosity': [1],
              'subsample': [0.5, 0.7, 1],
              'colsample_bytree': [1],
              'n_estimators': [500]}

reg_grid=sk.model_selection.GridSearchCV(reg, parameters, scoring='r2', n_jobs=None, refit=True, cv=None, verbose=2, return_train_score=False)

y_pred1=reg_grid.fit(x_train,y_train)

In [ ]:
#score=sk.model_selection.cross_val_score(reg_grid, x_train, y_train,cv=kfold)
#score.mean()